# Conexiones e importaciones

In [1]:
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify
import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)
import numpy as np
import mysql.connector
from mysql.connector import Error

In [2]:
load_dotenv() #carga las variables del entorno .env; devuelve un true o false

#Spotify
mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

#last fm
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

#MySQL
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")

# Spotify

In [ ]:
def busqueda_spotify(generos, año):
    todas_las_canciones = [] # guarda todas las canciones encontradas
    albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).

    print("Buscando canciones del año", año,"...\n")

    for genero in generos:
        print("Género:", genero)

        # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
        #    No funciona con álbumes ni canciones directamente.
        resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]

            # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
            #    al buscar álbumes (type="album") o canciones (type="track").
            #    Aquí buscamos álbumes del artista en el año indicado.
            busqueda = "artist:" + nombre_artista + " year:" + str(año)
            resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
            albumes = resultado_albumes["albums"]["items"]
            
            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]
                
                # Si ya vimos este álbum, saltamos al siguiente
                if id_album in albumes_ya_vistos:
                    continue
                
                # Si no lo habíamos visto, lo marcamos como visto
                albumes_ya_vistos.add(id_album)

                # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print(f"→ {len(genero)} Canciones de {genero} añadidas")
        print()

    # Mostramos un resumen de lo encontrado
    print("RESUMEN:")

    # Contamos las canciones de cada género una por una
    for genero in generos:
        contador = 0
        for cancion in todas_las_canciones:
            if cancion["genero"] == genero:
                contador = contador + 1
        print(f"- {genero}: {contador} canciones")

    print("\nTotal de canciones encontradas:", len(todas_las_canciones))

    todas_las_canciones_df = pd.DataFrame(todas_las_canciones)  
    return todas_las_canciones_df

In [4]:
genero = ["classical","latin","jazz","rock"]
año = 2010
canciones_2010_df = busqueda_spotify(genero,año)

Buscando canciones del año 2010
Género: classical
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- classical: 14539 canciones
- latin: 1500 canciones
- jazz: 3280 canciones
- rock: 4276 canciones
Total de canciones encontradas: 23595


In [5]:
canciones_2010_df

,nombre,artista,album,genero,año
0,The Planets - Live,Ludovico Einaudi,The Royal Albert Hall Concert,classical,2010
1,Lady Labyrinth - Live,Ludovico Einaudi,The Royal Albert Hall Concert,classical,2010
2,Nightbook - Live,Ludovico Einaudi,The Royal Albert Hall Concert,classical,2010
3,In Principio - Live,Ludovico Einaudi,The Royal Albert Hall Concert,classical,2010
4,Indaco - Live,Ludovico Einaudi,The Royal Albert Hall Concert,classical,2010
...,...,...,...,...,...
23590,El Amor Lo Cura Todo,Juanes,P.A.R.C.E.,rock,2010
23591,Todos Los Días,Juanes,P.A.R.C.E.,rock,2010
23592,Y No Regresas,Juanes,P.A.R.C.E.,rock,2010
23593,Lo Nuestro,Juanes,P.A.R.C.E.,rock,2010


In [7]:
canciones_2010_df.to_csv("canciones_2010.csv", index=False)

# Last FM

In [8]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [9]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/")

In [11]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no interfieran con la url de la api
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
                        'method': 'artist.getinfo',
                        'artist': artista_codificado,
                        'api_key': api_key_lastfm,
                        'format': 'json'
                    }
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "artist" in data: 
            artista_info = data['artist']
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()
            return {
                        'bio_resumen': bio_summary,
                        'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),
        }
        else:
            # Artista no encontrado por Last.fm
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Incluye HTTPError, ConnectionError, TimeoutError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = canciones_2010_df['artista'].unique() #extrae los artistas sin repetir
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista']) #df temporal con los artistas sin repetir
    
    print("\nIniciando consultas a Last.fm...")
    
    #apply pasa el valor de la columna "artista" como primer argumento posicional. Con "args" se pasan el resto de argumentos.
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista, 
        args=(api_key_lastfm,) 
    )

    # 4. Normalizar los resultados (convertir la Serie de Diccionarios a Columnas de DF)
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 120

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [12]:
datos_lastfm_df

,artista,bio_resumen,listeners
0,Ludovico Einaudi,Ludovico Maria Enrico Einaudi OMRI (born 23 No...,1764546
1,Yann Tiersen,"Yann Tiersen (born in Brest, Brittany, France ...",1907976
2,Franz Liszt,Franz Liszt (22 October 1811 – 31 July 1886) w...,1351008
3,Ólafur Arnalds,Born 1986 in the suburban Icelandic town of Mo...,1189205
4,Erik Satie,Eric Alfred Leslie Satie (17 May 1866 – 1 July...,1802568
...,...,...,...
115,Maroon 5,Maroon 5 is an American pop rock band that ori...,6442262
116,Imagine Dragons,Imagine Dragons is an American pop band formed...,3948207
117,Elton John,Sir Elton Hercules John (born Reginald Kenneth...,5080474
118,Maná,There is more than one artist with this name:\...,818403


# Creación BD

In [13]:
try:
    cnx = mysql.connector.connect(
        host= MYSQL_HOST,
        user= MYSQL_USER,
        password= MYSQL_PASSWORD,
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS MusicStream_db"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Conexión exitosa
Query exitosa


In [17]:
mycursor.execute("USE MusicStream_db")

# se añaden en varios execute porque a todas se les ha llamado query, por lo que reemplaza a la anterior y solo ejecuta la última válida

query = '''CREATE TABLE IF NOT EXISTS Canciones(
	ID_Cancion INT AUTO_INCREMENT,
    Nombre TEXT NOT NULL,
    Artista VARCHAR(45),
    Album TEXT,
    Genero VARCHAR(30),
    Lanzamiento YEAR,
    PRIMARY KEY (ID_Cancion)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS Artistas(
    ID_Artista INT AUTO_INCREMENT,
    Artista VARCHAR(70) UNIQUE NOT NULL,
    Info TEXT,
    Oyentes INT,
    PRIMARY KEY (ID_Artista)
);'''
mycursor.execute(query)

In [18]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Canciones (Nombre, Artista, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s, %s)
"""

try:
    df_corregido_SPOTIFY = canciones_2010_df.replace({np.nan: None, 'nan': None, 'NaN': None}) # corregimos valores nulos para que MySQL los entienda
    datos = df_corregido_SPOTIFY.values.tolist()

    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit() #indispensable para guardar los cambios y que se complete la petición hecha (sio da algún error antes del commit se quedan ambos programas en standby)

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback() # revertir petición en caso de error
    

23595 registros insertados


In [19]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Artistas (Artista, Info, Oyentes) 
VALUES (%s, %s, %s)
"""

try:
    df_corregido_LASTFM = datos_lastfm_df.replace({np.nan: None, 'nan': None, 'NaN': None})
    datos = df_corregido_LASTFM.values.tolist()
    
    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit()

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback()

120 registros insertados


In [ ]:
cnx.close()